In [ ]:
%matplotlib inline
%matplotlib widget

import subprocess as sp

from onset_detection import *
from onset_detection.audio import *
from onset_detection.od import *


---

# Remixing
This jupyter notebook allows the remixing of manually tapped rhythms back into its original track

### Pick a song
Choose a song you like from the list of songs below and listen back to it if you want:

- The Beatles - In My Life: dict key = `"beatles"`
- Queen - Another One Bites the Dust: dict key = `"queen"`
- Arctic Monkeys - The View From the Afternoon: dict key = `"arctic_monkeys"`
- Queens of the Stone Age - No One Knows: dict key = `"qotsa"`
- Slow J - Mundança: dict key = `"slowj"`
- AC/DC - Back in Black: dict key = `"ac_dc"`

In [ ]:
song = "qotsa"
trim = False

In [ ]:
song_path = song_dict[song]
trim_interval = None
# choose the interval in which you want to trim the audio (in seconds)
if trim:
    trim_interval = (70, 95)

song_audio, song_sr = read_audio(song_path, trim_interval)

In [ ]:
ipd.Audio(song_audio, rate=song_sr)

### Separating the drums from the rest

Using S. Rouard et al.'s [Demucs](https://github.com/facebookresearch/demucs) [2], we can separate the drums from the rest of the other intruments:

In [ ]:
output_dir = "../results/demucs"

In [ ]:
separate_drums = sp.run(["python3.9", "-m", "demucs", "--two-stems=drums", song_path, "-o", output_dir])

Listen back to the isolated drums, or to the original track without them

In [ ]:
# Drums only
song_name = song_path.split('/')[-1].split('.')[0]
drums_only = output_dir + "/mdx_extra_q/" + song_name + "/drums.wav"

drums_audio, drums_sr = read_audio(drums_only, trim_interval)

In [ ]:
ipd.Audio(drums_audio, rate=drums_sr)

In [ ]:
# No drums
no_drums = output_dir + "/mdx_extra_q/" + song_name + "/no_drums.wav"

no_drums_audio, no_drums_sr = read_audio(no_drums, trim_interval)

In [ ]:
ipd.Audio(no_drums_audio, rate=no_drums_sr)

### Tapped rhythms version
Listen back to tapped rhythms version of the track you picked

In [ ]:
tapped_path = tapped_dict[song]
tapped_audio, tapped_sr = read_audio(tapped_path, trim_interval)

In [ ]:
ipd.Audio(tapped_audio, rate=tapped_sr)

### Remixing the tapped rhythms into the original song

Now we can replace the original drums with our tapped version of them!

In [ ]:
# the tapped rhythms are usually lower in volume, so we need to upmix them a bit to make the whole mix clearer sounding
mixing_factor = 15
tapped_version = no_drums_audio + tapped_audio * mixing_factor

In [ ]:
ipd.Audio(tapped_version, rate=tapped_sr)

### MIDI Onset Detection(OD)
We can also generate a MIDI file from the tapped rhythms version by using `madmom`'s  [Onset Detection(OD) module](https://madmom.readthedocs.io/en/v0.16/modules/features/onsets.html)

First, let's calculate the Detection Function(DF) using J. Schlüter and S. Böck's CNN method [1]:

In [ ]:
df = cnn_od(tapped_audio)
df_bins = df.size

Next, we need to process this DF and identify which peaks are actually onsets:

In [ ]:
b_df = peak_picking(df)

Finally, we calculate the onset times from the peaks and we generate a MIDI file from the onsets

In [ ]:
times = calculate_onset_times(tapped_audio, tapped_sr, b_df, df_bins)
midi_path = od2midi(tapped_path, df, times)

In [ ]:
#midi_path = '../results/demucs/mdx_extra_q/qotsa-no_one_knows/qotsa_no_one_knows-tapped.wav'
midi_audio, midi_sr = read_audio(midi_path, trim_interval)
print(midi_audio)

In [ ]:
ipd.Audio(midi_audio, rate=midi_sr)

We can also remix this OD MIDI version back into our original track:

In [ ]:
# we also need to upmix the MIDI output
mixing_factor = 15
pad_width = no_drums_audio.size - midi_audio.size
midi_audio_padded = np.pad(midi_audio, (0, pad_width), 'constant')

midi_version = no_drums_audio + midi_audio_padded * mixing_factor

In [ ]:
 ipd.Audio(midi_version, rate=midi_sr)

---

## Bibliography

[1] [J. Schlüter and S. Böck, ‘Improved musical onset detection with Convolutional Neural Networks’, *2014 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP)*, pg. 6979–6983, 2014.](https://ieeexplore.ieee.org/document/6854953)
[2] [S. Rouard, F. Massa, and A. Défossez, ‘Hybrid Transformers for Music Source Separation’. *arXiv*, 2022.](https://arxiv.org/abs/2211.08553)